# Binoxxo Puzzle

Lucerne University of Applied Sciences and Arts - School of Information Technology

Place X or O in the empty cells so that there are no more than two consecutive X's or O's in a row or a column.
The number of X's is the same as the number of O's in each row and column, and all rows and columns are unique.

Find more Binoxxo puzzles [here](https://www.binoxxo.ch/Binoxxo-Raetselbuch/)

In [84]:
%pip install ortools

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [85]:
from ortools.sat.python import cp_model
from itertools import product

Binoxxo puzzle from lecture

In [86]:
binoxxo1 = [
    ["", "1", "", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["", "1", "1", "", "", "", "", "", "", ""],
    ["", "", "", "", "0", "0", "", "", "", "0"],
    ["1", "", "", "", "", "", "1", "1", "", ""],
    ["", "1", "", "", "1", "", "", "", "", ""],
    ["", "", "", "0", "", "", "1", "", "", ""],
    ["", "0", "", "", "", "", "", "0", "", "0"],
    ["", "", "", "", "0", "", "", "", "", ""],
    ["0", "", "", "", "", "", "", "", "", "0"]
]

And two more examples

In [87]:
binoxxo2 = [
    ["", "", "0", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "", ""],
    ["0", "", "", "", "1", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", ""],
    ["0", "", "", "0", "", "", "", "", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "", "", "", "", "", "1", "1", ""],
    ["1", "", "", "", "", "", "", "1", "", ""],
    ["", "", "", "", "", "0", "", "", "", "0"],
    ["", "", "", "", "1", "", "0", "", "", ""]
]
binoxxo3 = [
    ["1", "", "", "", "", "", "0", "0", "", ""],
    ["", "", "", "0", "", "", "", "", "1", ""],
    ["", "", "0", "", "", "", "", "", "", ""],
    ["", "", "", "0", "", "1", "", "", "", "0"],
    ["1", "", "1", "", "", "1", "", "", "1", ""],
    ["", "", "1", "", "1", "", "", "", "", ""],
    ["", "", "", "", "1", "1", "", "", "1", ""],
    ["", "", "", "", "", "", "", "", "", "0"],
    ["", "1", "", "1", "", "", "", "", "", ""],
    ["", "1", "", "", "1", "", "", "1", "1", ""]
]

Pick one of the examples

In [88]:
game = binoxxo2

Create model

In [89]:
model = cp_model.CpModel()

In [90]:
boardSize = len(game)

sameAmountOfXsAndOs = boardSize // 2

board = [[model.NewIntVar(0, 1, f"({i},{j})") for j in range(boardSize)] for i in range(boardSize)]

# Set values for the given Binoxxo game
for i in range(boardSize):
    for j in range(boardSize):
        if binoxxo2[i][j] in {"0", "1"}:
            model.Add(board[i][j] == int(binoxxo2[i][j]))

# Rows and columns cannot be identical
for i in range(boardSize):
    for j in range(i + 1, boardSize):
        diff_vars_row = [model.NewBoolVar(f"diff_row_{i}_{j}_{k}") for k in range(boardSize)]
        diff_vars_col = [model.NewBoolVar(f"diff_col_{i}_{j}_{k}") for k in range(boardSize)]
        for k in range(boardSize):
            model.Add(board[i][k] != board[j][k]).OnlyEnforceIf(diff_vars_row[k])
            model.Add(board[i][k] == board[j][k]).OnlyEnforceIf(diff_vars_row[k].Not())
            model.Add(board[k][i] != board[k][j]).OnlyEnforceIf(diff_vars_col[k])
            model.Add(board[k][i] == board[k][j]).OnlyEnforceIf(diff_vars_col[k].Not())
        model.Add(sum(diff_vars_row) > 0)
        model.Add(sum(diff_vars_col) > 0)


# No more than two consecutive Xs (1s) or Os (0s) in any row or column
for i in range(boardSize):
    for j in range(boardSize - 2):
        model.Add(sum(board[i][k] for k in range(j, j + 3)) < 3)
        model.Add(sum(board[i][k] for k in range(j, j + 3)) > 0)
        model.Add(sum(board[k][i] for k in range(j, j + 3)) < 3)
        model.Add(sum(board[k][i] for k in range(j, j + 3)) > 0)

# Each row and column must have an equal number of Xs (1s) and Os (0s)
for i in range(boardSize):
    model.Add(sum(board[i][j] for j in range(boardSize)) == sameAmountOfXsAndOs)
    model.Add(sum(board[j][i] for j in range(boardSize)) == sameAmountOfXsAndOs)


Callback for solution printing

In [91]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables

    def on_solution_callback(self):
        mapper = {0:'O', 1:'X'}
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{mapper[self.Value(self.__variables[i][j])]}] ", end='')
            print("\n")
        print("\n")

Solve and print all solutions

In [92]:
solver = cp_model.CpSolver()
solver.parameters.enumerate_all_solutions = True
status = solver.Solve(model, SolutionPrinter(board))

[O] [X] [O] [O] [X] [X] [O] [O] [X] [X] 

[X] [O] [X] [O] [O] [X] [X] [O] [O] [X] 

[O] [X] [O] [X] [X] [O] [X] [X] [O] [O] 

[X] [O] [O] [X] [X] [O] [O] [X] [X] [O] 

[O] [X] [X] [O] [O] [X] [X] [O] [O] [X] 

[X] [O] [X] [O] [O] [X] [X] [O] [X] [O] 

[O] [X] [O] [X] [X] [O] [O] [X] [X] [O] 

[X] [O] [X] [O] [O] [X] [O] [X] [O] [X] 

[X] [O] [X] [X] [O] [O] [X] [O] [X] [O] 

[O] [X] [O] [X] [X] [O] [O] [X] [O] [X] 





In [94]:
print(f"Runtime:  {solver.WallTime()}ms")

Runtime:  0.025387000000000003ms
